# Word Embedding

> the collective name for a set of language modeling and feature learning techniques in NLP where words and phrases from the vocabulary are mapped to vectors of real numbers. (from Wikipedia)

> 수학적으로, 고차원의 공간을 더 낮은 공간으로 변환하는 방법(embedding)과 같은 의미이기도 하다.

> 결국, 고차원으로 표현된 feature vector(local representation, BOW, TF-IDF 등)을 distributional semantic을 가지는 vector space에 mapping 시켜주는 방법이다.

> <b>"You shall know a word by the company it keeps"(John R. Firth, 1957)<b>, it called "Distributed Hypothesis"

![word embedding fig](figs/word-vector-space-similar-words.png)

![visualize word vectors](figs/visualize-word-vectors.png)

다음은 최근 많이 쓰이는 word embedding 방법들이다.

wevi : word embedding visual inspector
    
> https://ronxin.github.io/wevi/

이론을 정리하기 위해 체험을 해보자.

## 1. Word2Vec

> 현재 word embedding이 핫하게 된 시작 알고리즘. "Distributed representations of words and phrases and their compositionality(NIPS 2013)" 에 처음 소개되었다.

> Reference : https://code.google.com/archive/p/word2vec/

![skip-gram](figs/skip-gram.png)
![simple-skip-gram](figs/simple-skip-gram.png)

> Skip-Gram with Negative Sampling, 줄여서 SGNS라고 부르며 Neural Net을 이용한 word embedding이 빠르게 구현가능해진 이유기도 하다.

> Negative Sampling이란, 마지막 단계의 softmax를 구하는 문제를 주변 단어(postive class)와 무작위로 골라진 나머지 단어들(negative class)로 분류하는 binary classfication 문제로 바꿔주는 기법이며 이를 통해 굉장히 빠르게 word embedding 수행이 가능하다.

In [1]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import movie_reviews
sentences = [list(sent) for sent in movie_reviews.sents()]
sentences[0]

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.']

In [3]:
# gensim에서 word2vec 불러오기
from gensim.models import Word2Vec

# 학습 모델 구현.
w2v_model = Word2Vec(sentences, min_count=5, size=300, sg=1, iter=10, workers=4, ns_exponent=0.75, window=7)  # sg=1 => Skip Gram을 사용하겠다. / workers=4 => CPU를 4개 정도 사용하겠다.

In [4]:
# 모델 평가, 비슷한 단어 찾기.
w2v_model.wv.most_similar("good")

[('decent', 0.5294654369354248),
 ('darned', 0.48018771409988403),
 ('meaty', 0.4768997132778168),
 ('lousy', 0.4737260937690735),
 ('dopey', 0.47170382738113403),
 ('milestone', 0.4691770374774933),
 ('gutsy', 0.466529905796051),
 ('fantastic', 0.4641210436820984),
 ('commendable', 0.44876447319984436),
 ('crappy', 0.4473940134048462)]

## 2. GloVe

> GloVe는 Gloval Vectors의 약자로, aggregated global word co-occurence statistics를 최적화하는 방향으로 학습하는 word embedding 방법이다. "GloVe: Gloval Vectors for Word Representation(EMNLP 2014)"에 소개되었다.

> Reference : https://nlp.stanford.edu/projects/glove/

![glove](figs/glove.png)

In [5]:
# glove는 현재 파이썬으로는 구현이 힘듭니다, 사용을 위해서는 c++을 사용하셔야 합니다.
#from glove import Glove

#glove_model = Glove(no_components=100, learning_rate=0.05)
#glove_model.fit(sentences, epochs=10, no_threads=4, verbose=True)

## 3. FastText

> 현재 NLP task에서 word embedding의 baseline으로 사용되는 기법이다. subword embedding model, char n-gram embedding model이라고도 한다.

> word2vec을 만들었던, Tomas Mikolov가 Google에서 Facebook으로 옮긴 뒤에 낸 모델로 word2vec의 단점을 보완한 모델이다.

> word2vec의 단점이었던, OOV 문제와 low frequency를 많이 해결하였다.

> word를 subword 단위로 표현하는 것으로 기본적으로 SGNS 방식이다.

> Reference : https://fasttext.cc/

![char3-grams](figs/char3-grams.png)

In [6]:
# gensim에서 FastText 불러오기
from gensim.models import FastText

# FastText 학습.
fast_model = FastText(sentences, min_count=5, sg=1, size=300, workers=4, min_n=2, max_n=7, alpha=0.05, iter=10, window=7)
fast_model.save("fast_model")

In [7]:
# 결과 평가.
fast_model.wv.most_similar("good")

[('goods', 0.49452564120292664),
 ('goodnight', 0.41828882694244385),
 ('goofball', 0.4050053358078003),
 ('goose', 0.391421914100647),
 ('great', 0.3896257281303406),
 ('goo', 0.3877134919166565),
 ('bad', 0.3793599307537079),
 ('goop', 0.36883866786956787),
 ('decent', 0.36726880073547363),
 ('excellent', 0.365870863199234)]

## 4. ELMo

![elmo_sesame_street](figs/elmo_sesame_street.png)

> ELMo는 Embeddings from Language Model의 약자입니다. ELMo는 pre-trained language model을 사용하여 문맥에 맞는 word embedding, "Contextualized Word Embedding"을 만드는 방법입니다.

> bidirectional Language Model을 이용하여, pre-trained embedding vector를 corpus의 context(syntax, semantics, polysemy) 정보를 보완해주는 embedding vector를 만들어 준다.

> tensorflow, pytorch를 통해서 bidirectional LSTM model을 만들어 사용이 가능하다. (이미 구현된 model이 github에 공개되어있다.)

> "Deep contextualized word representations(NAACL 2018)"에 소개된 방법입니다.

In [8]:
#!pip install tensorflow_hub

In [9]:
# # tensorflow_hub에서 데이터 불러오기.
# import tensorflow_hub as hub

# # 가까운 미래에는 실행이 되길 빌면서..
# elmo = hub.load("https://tfhub.dev/google/elmo/3")

# embeddings = elmo(
#     ["the cat is on the mat", "dogs are in the fog"],
#     signature="default",
#     as_dict=True)["elmo"]

In [10]:
# print(embeddings)

> Reference : https://allennlp.org/elmo, https://github.com/allenai/bilm-tf

![elmo_architecture](figs/elmo_structure.png)

![elmo_model](figs/elmo_model.png)